In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
# from llama_index import VectorStoreIndex,SimpleDirectoryReader
# import docsearch
import pinecone

c:\Users\NH2854\medbot2\medchat\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
CTransformers

langchain.llms.ctransformers.CTransformers

In [4]:
PINECONE_API_KEY = "274c03e9-281b-46bc-a8c1-852d01185b0f"
PINECONE_API_ENV = "gcp-starter"

In [5]:
import os

current_directory = os.getcwd()
data_path = os.path.join(current_directory, "data/")


In [6]:
data_path

'c:\\Users\\NH2854\\medbot2\\data/'

In [7]:
  #Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader) 
    documents = loader.load()
    return documents  

In [8]:
extracted_data = load_pdf("data/")
#extracted_data = load_pdf(r"C:\Users\NH2854\Medical-chatbot\data\Medical_book.pdf")


In [9]:
type(extracted_data)

list

In [10]:
#Create text chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    # text_chunks = text_splitter.split_documents(extracted_data)

    # return tuple(text_chunks)

In [11]:
# text_chunks = text_split(extracted_data)
# print("length of my chunk:", len(text_chunks))

In [12]:
# type(text_chunks)

In [13]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [14]:
embeddings = download_hugging_face_embeddings()

In [15]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [16]:
# query_result = embeddings.embed_query("Hello world")
# print("Length", len(query_result))

In [17]:
from pinecone import Pinecone

pc = Pinecone(api_key="c727e84c-88ca-4752-aea8-a70fbedf5980")
index = pc.Index("medical-chatbot")

In [18]:

# Specify the index name
index_name = "medical-chatbot"

# index=Pinecone.index('medical-chatbot')

In [19]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.14375,
 'namespaces': {'': {'vector_count': 14375}},
 'total_vector_count': 14375}

In [20]:
# from tqdm.auto import tqdm
# from uuid import uuid4

# batch_limit = 100

# texts = []
# metadatas = []

# # Assuming 'data' is the list of Document objects
# for i, document in enumerate(tqdm(extracted_data)):
#     # Extract metadata from the document
#     metadata = {
#         'source': document.metadata['source'],
#         'page no': document.metadata['page']
#     }
#     # Extract text content from the document
#     text = document.page_content  # Assuming 'page_content' contains the text content
#     # Now we create chunks from the text content
#     record_texts = text_splitter.split_text(text)
#     # Create individual metadata dicts for each chunk
#     record_metadatas = [{
#         "chunk": j, "text": text, **metadata
#     } for j, text in enumerate(record_texts)]
#     # Append these to current batches
#     texts.extend(record_texts)
#     metadatas.extend(record_metadatas)
#     # If we have reached the batch_limit we can add texts
#     if len(texts) >= batch_limit:
#         ids = [str(uuid4()) for _ in range(len(texts))]
#         embeds = embeddings.embed_documents(texts)
#         index.upsert(vectors=zip(ids, embeds, metadatas))
#         texts = []
#         metadatas = []

# # Process the remaining texts if any
# if len(texts) > 0:
#     ids = [str(uuid4()) for _ in range(len(texts))]
#     embeds = embeddings.embed_documents(texts)
#     index.upsert(vectors=zip(ids, embeds, metadatas))


In [21]:
# texts = [
#     'this is the first chunk of text',
#     'then another second chunk of text is here'
# ]

# res = embeddings.embed_documents(texts)
# len(res), len(res[0])

In [22]:
# import time

# index_name = 'medical-chatbot'
# existing_indexes = [
#     index_info["name"] for index_info in pc.list_indexes()
# ]

# # check if index already exists (it shouldn't if this is first time)
# if index_name not in existing_indexes:
#     # if does not exist, create index
#     pc.create_index(
#         index_name,
#         dimension=384,  # dimensionality of ada 002
#         metric='dotproduct',
#         spec=spec
#     )
#     # wait for index to be initialized
#     while not pc.describe_index(index_name).status['ready']:
#         time.sleep(1)

# # connect to index
# index = pc.Index(index_name)
# time.sleep(1)
# # view index stats
# index.describe_index_stats()

In [23]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.14375,
 'namespaces': {'': {'vector_count': 14375}},
 'total_vector_count': 14375}

In [24]:
from langchain.vectorstores import Pinecone

text_field = "page_content"  # the metadata field that contains our text

# switch back to normal index for langchain
index = pc.Index(index_name)

# initialize the vector store object
vectorstore = Pinecone(
    index, embeddings.embed_query, text_field
)

In [25]:
model="sentence-transformers/all-MiniLM-L6-v2"

In [26]:

from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)
model
     

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [63]:

query = "what is acne?"

# create the query vector
xq = model.encode(query).tolist()
# now query
xc = index.query(vector=xq, top_k=5, include_metadata=False)
xc

NameError: name 'res' is not defined

In [47]:
limit = 3750

import time

def retrieve(query):
    res =embeddings

    # retrieve from Pinecone
    xq = res['data'][0]['embedding']

    # get relevant contexts
    contexts = []
    time_waited = 0
    while (len(contexts) < 3 and time_waited < 60 * 12):
        res = index.query(vector=xq, top_k=3, include_metadata=True)
        contexts = contexts + [
            x['metadata']['text'] for x in res['matches']
        ]
        print(f"Retrieved {len(contexts)} contexts, sleeping for 15 seconds...")
        time.sleep(15)
        time_waited += 15

    if time_waited >= 60 * 12:
        print("Timed out waiting for contexts to be retrieved.")
        contexts = ["No contexts retrieved. Try to answer the question yourself!"]


    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt


def complete(prompt):
    # instructions
    sys_prompt = "You are a helpful assistant that always answers questions."
    # query text-davinci-003
    qa=RetrievalQA.from_chain_type(
            llm=llm, 
            chain_type="stuff",
            retriever=vectorstore.as_retriever(),
             temperature=0

               
    )
    return qa['choices'][0]['message']['content'].strip()

TypeError: 'HuggingFaceEmbeddings' object is not subscriptable

In [29]:

for result in xc['matches']:
    print(f"{round(result['score'], 5)}: {result['metadata']['text']}")
     

PineconeApiAttributeError: ScoredVector has no attribute 'metadata' at ['['received_data', 'matches', 0]']['metadata']

In [31]:

query = "which metropolis has the highest number of people?"

# create the query vector
xq = model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=5, include_metadata=True)
for result in xc['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")
     

0.32: Between two and four million Americans have AD;
0.32: Between two and four million Americans have AD;
0.31: Numerous epidemiological studies of populations are
0.31: Numerous epidemiological studies of populations are
0.27: and so there is a higherproportion of women in the most affected age groups.


In [32]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [33]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [34]:
checkpoint= "model"
llm=CTransformers(model="model",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [35]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    
  )

In [58]:
def complete(prompt):
    # instructions
    sys_prompt = "You are a helpful assistant that always answers questions."
    # query text-davinci-003
    qa=RetrievalQA.from_chain_type(
            llm=llm, 
            chain_type="stuff",
            retriever=vectorstore.as_retriever(),
                  
    )
    return qa['choices'][0]['message']['content'].strip()

In [59]:
complete(query)

TypeError: 'RetrievalQA' object is not subscriptable